# Steam:Big Data
Objective: 
- Use Big Data tools like Databricks and PySpark to conduct an analysis in order to understand the factors that affect sales and drive the video game industry.
- The performed analysis will be divided in three different parts: macro-level, genres and platform analysis.

  - Macro-level analysis:
    - Which publisher has released the most games on Steam?

    - What are the best rated games?

    - Are there years with more releases? Were there more or fewer game releases during the Covid, for example?

    - How are the prices distributed? Are there many games with a discount?

    - What are the most represented languages?

    - Are there many games prohibited for children under 16/18?
  - Genres analysis:
  
    - What are the most represented genres?
    
    - Are there any genres that have a better positive/negative review ratio?
    
    - Do some publishers have favorite genres?
    
    - What are the most lucrative genres?

  - Platform Analysis:
    - Are most games available on Windows/Mac/Linux instead?

    - Do certain genres tend to be preferentially available on certain platforms?

## Imports

In [ ]:
# libraries
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark

In [ ]:
# import data from S3 bucket into pyspark dataframe
path = "s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json"
df = spark.read.format("json").option("header", "true").load(path)

# view schema
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

- It is a nested schema, so it will be necessary to do some tidying up and transformations before getting to analysis.

## Basic Exploration

In [ ]:
# show top 5 rows
df.show(5)

+--------------------+-------+
|                data|     id|
+--------------------+-------+
|{10, [Multi-playe...|     10|
|{1000000, [Single...|1000000|
|{1000010, [Single...|1000010|
|{1000030, [Multi-...|1000030|
|{1000040, [Single...|1000040|
+--------------------+-------+
only showing top 5 rows



In [ ]:
# show one sample
df.take(1)

Out[4]: [Row(data=Row(appid=10, categories=['Multi-player', 'Valve Anti-Cheat enabled', 'Online PvP', 'Shared/Split Screen PvP', 'PvP'], ccu=13990, developer='Valve', discount='0', genre='Action', header_image='https://cdn.akamai.steamstatic.com/steam/apps/10/header.jpg?t=1666823513', initialprice='999', languages='English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean', name='Counter-Strike', negative=5199, owners='10,000,000 .. 20,000,000', platforms=Row(linux=True, mac=True, windows=True), positive=201215, price='999', publisher='Valve', release_date='2000/11/1', required_age='0', short_description="Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.", tags=Row(1980s=266, 1990's=1

In [ ]:
type(df)

Out[5]: pyspark.sql.dataframe.DataFrame

In [ ]:
# length of dataframe
df.count()

Out[6]: 55691

## Exploratory Data Analysis

### Macro-Level Analysis
 - Which publisher has released the most games on Steam?

  - What are the best rated games?

  - Are there years with more releases? Were there more or fewer game releases during the Covid, for example?

  - How are the prizes distributed? Are there many games with a discount?

  - What are the most represented languages?

  - Are there many games prohibited for children under 16/18?

In [ ]:
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

### Which publisher has released the most games on Steam?

In [ ]:
# show top 5 publishers

# group data by publisher count and order descendingly, by count
df.groupBy("data.publisher").count().orderBy(desc("count")).show(5)

+--------------+-----+
|     publisher|count|
+--------------+-----+
|Big Fish Games|  422|
|        8floor|  202|
|          SEGA|  165|
|Strategy First|  151|
|   Square Enix|  141|
+--------------+-----+
only showing top 5 rows



### What are the best rated games?

### Are there years with more releases? Were there more or fewer game releases during the Covid, for example?

In [ ]:
# group data by release date
df.groupBy("data.release_date").count().orderBy(desc("count")).show(5)

+------------+-----+
|release_date|count|
+------------+-----+
|            |   99|
|  2020/01/17|   74|
|  2022/09/30|   64|
|  2020/10/15|   63|
|  2021/09/30|   62|
+------------+-----+
only showing top 5 rows



In [ ]:
# select date column and convert it to DateType
release_date = df.select("data.release_date")
release_date = release_date.withColumn("release_date", col("release_date").cast(DateType()))
release_date.printSchema()

root
 |-- release_date: date (nullable = true)



In [ ]:
# split data in "release_date column", where parentheses is located
release_explode = df.withColumn("release_date", split(df["data.release_date"], "/"))
release_explode.show(5)

+--------------------+-------+--------------+
|                data|     id|  release_date|
+--------------------+-------+--------------+
|{10, [Multi-playe...|     10| [2000, 11, 1]|
|{1000000, [Single...|1000000|[2021, 05, 14]|
|{1000010, [Single...|1000010|[2020, 10, 16]|
|{1000030, [Multi-...|1000030|[2020, 10, 14]|
|{1000040, [Single...|1000040|[2019, 03, 30]|
+--------------------+-------+--------------+
only showing top 5 rows



In [ ]:
# explode "release_date" column
# it is necessary to remove day and months
df_date_explode = release_explode.withColumn("release_date", explode("release_date"))
df_date_explode.show(5)

+--------------------+-------+------------+
|                data|     id|release_date|
+--------------------+-------+------------+
|{10, [Multi-playe...|     10|        2000|
|{10, [Multi-playe...|     10|          11|
|{10, [Multi-playe...|     10|           1|
|{1000000, [Single...|1000000|        2021|
|{1000000, [Single...|1000000|          05|
+--------------------+-------+------------+
only showing top 5 rows



In [ ]:
# keep records where length of release date is 4, in order to remove days and months
df_date_explode = df_date_explode.where(length("release_date") == 4)
df_date_explode.show(5)

+--------------------+-------+------------+
|                data|     id|release_date|
+--------------------+-------+------------+
|{10, [Multi-playe...|     10|        2000|
|{1000000, [Single...|1000000|        2021|
|{1000010, [Single...|1000010|        2020|
|{1000030, [Multi-...|1000030|        2020|
|{1000040, [Single...|1000040|        2019|
+--------------------+-------+------------+
only showing top 5 rows



In [ ]:
# count number of records per year
counts_release_date = df_date_explode.groupBy("release_date").count().orderBy(desc("count"))
counts_release_date.show(50)

+------------+-----+
|release_date|count|
+------------+-----+
|        2021| 8823|
|        2020| 8305|
|        2018| 7678|
|        2022| 7455|
|        2019| 6968|
|        2017| 6017|
|        2016| 4185|
|        2015| 2576|
|        2014| 1557|
|        2013|  471|
|        2012|  345|
|        2009|  311|
|        2010|  288|
|        2011|  267|
|        2008|  159|
|        2007|   98|
|        2006|   61|
|        2005|    6|
|        2004|    6|
|        2001|    4|
|        1999|    3|
|        2003|    3|
|        2000|    2|
|        1997|    2|
|        2002|    1|
|        1998|    1|
+------------+-----+



In [ ]:
# viz
display(counts_release_date)

release_date,count
2021,8823
2020,8305
2018,7678
2022,7455
2019,6968
2017,6017
2016,4185
2015,2576
2014,1557
2013,471


Databricks visualization. Run in Databricks to view.

##### Insight
- The year with most releases? 2021
- Covid did not have any effect on releases, in fact 2021 was the year with the most of them
- Between 2014 and 2017 the growth rate was around 50% of releases between each year

### How are the prices distributed? Are there many games with a discount?

#### Prices Distribution

In [ ]:
# select price column and convert it to integer
price = df.select("data.price")
price = price.withColumn("price", col("price").cast(IntegerType()))
price.printSchema()

root
 |-- price: integer (nullable = true)



In [ ]:
# statistical summary
price.describe().show()

+-------+-----------------+
|summary|            price|
+-------+-----------------+
|  count|            55691|
|   mean|773.2849832109317|
| stddev| 1093.13458272345|
|    min|                0|
|    max|            99900|
+-------+-----------------+



In [ ]:
# highest priced game
max_price = price.select(max("price"))
max_price.show()

+----------+
|max(price)|
+----------+
|     99900|
+----------+



In [ ]:
# what is the name of the game that costs $99,900?
# use filter and fetch name that matches conditions
df.filter(df["data.price"] == "99900").select("data.name").show(20, False)

+---------------------------------+
|name                             |
+---------------------------------+
|Ascent Free-Roaming VR Experience|
+---------------------------------+



##### Insight
- Average game costs $773
- 20% of games cost less than $100
- 50% of games cost less than $500
- 90% of games cost less than $2,000
- Surprisigly there is a game that costs $99.900. It is called "Ascent Free-Roaming VR Experience".

#### Discounts

In [ ]:
df.select("data.discount").describe().show()

+-------+------------------+
|summary|          discount|
+-------+------------------+
|  count|             55691|
|   mean| 2.603777989262179|
| stddev|12.887080174743176|
|    min|                 0|
|    max|                90|
+-------+------------------+



In [ ]:
# select discount column and convert it to integer
discount = df.select("data.discount")
discount = discount.withColumn("discount", col("discount").cast(IntegerType()))
discount.printSchema()

root
 |-- discount: integer (nullable = true)



In [ ]:
# what is the average for those games that have discounts?
# exclude games with no discount (0)
discount.filter(discount["discount"] > 0).describe().show()

+-------+------------------+
|summary|          discount|
+-------+------------------+
|  count|              2518|
|   mean| 57.58816521048451|
| stddev|22.512945353983127|
|    min|                10|
|    max|                90|
+-------+------------------+



##### Insight
- 95.48% of games do not have discounts
- The mean discount for those games that do have one is of 57.58%, the minimum being 10% and max 90%

### What are the most represented languages?

In [ ]:
# group data by most appearances of languages
df.groupBy("data.languages").count().orderBy(desc("count")).show(5, False)

+------------------------------------------+-----+
|languages                                 |count|
+------------------------------------------+-----+
|English                                   |29163|
|English, Russian                          |1881 |
|English, Simplified Chinese               |1200 |
|English, Japanese                         |1015 |
|English, Not supported, Simplified Chinese|683  |
+------------------------------------------+-----+
only showing top 5 rows



In [ ]:
# convert "languages" column to array, separate where comma is
# in order to explode the column
language_explode = df.withColumn("language", split(df["data.languages"], ","))
language_explode.show(5)

+--------------------+-------+--------------------+
|                data|     id|            language|
+--------------------+-------+--------------------+
|{10, [Multi-playe...|     10|[English,  French...|
|{1000000, [Single...|1000000|[English,  Korean...|
|{1000010, [Single...|1000010|[Simplified Chine...|
|{1000030, [Multi-...|1000030|           [English]|
|{1000040, [Single...|1000040|[Simplified Chinese]|
+--------------------+-------+--------------------+
only showing top 5 rows



In [ ]:
# explode "language" column
df_lang_explode = language_explode.withColumn("language", explode("language"))
df_lang_explode.printSchema()
df_lang_explode.show()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

##### Insight
- English has 4 times more appearances than the next language!
- Top 10 languages:

In [ ]:
# show top 10 languages
df_lang_explode.groupBy("language").count().orderBy(desc("count")).show(10)

+--------------------+-----+
|            language|count|
+--------------------+-----+
|             English|54646|
|              German|13996|
|              French|13406|
|             Russian|12839|
|     Spanish - Spain|12224|
|  Simplified Chinese|12213|
|            Japanese|10170|
|             Italian| 9297|
| Portuguese - Brazil| 6739|
|              Korean| 6575|
+--------------------+-----+
only showing top 10 rows



### Are there many games prohibited for children under 16/18?

In [ ]:
# select required_age column and convert it to integer
required_age = df.select("data.required_age")
required_age = required_age.withColumn("required_age", col("required_age").cast(IntegerType()))
required_age.printSchema()

root
 |-- required_age: integer (nullable = true)



In [ ]:
# statistics
required_age.describe()

Out[46]: DataFrame[summary: string, required_age: string]

In [ ]:
# amount of games that require an age higher than ages 16 and 18

# for loop, for selected ages append to empty dict the quantity of games having that age restriction
ages = [16, 18]
required_age_dict = {}

for age in ages:
    required_age_dict[age] = required_age.filter(required_age["required_age" ] > age).count()

display(required_age_dict)

{16: 267, 18: 6}

##### Insight
- 98.82% of games do not have age restriction, they are for everyone as it is 0.
- 267 games require an age higher than 16, while only 6 require an age higher than 18.

### Genres Analysis
- What are the most represented genres?

- Are there any genres that have a better positive/negative review ratio?

- Do some publishers have favorite genres?

- What are the most lucrative genres?

In [ ]:
# explode "genre" column
genre_explode = df.withColumn("genre", split(df["data.genre"], ","))
df_genre_explode = genre_explode.withColumn("genre", explode("genre"))
df_genre_explode.printSchema()
df_genre_explode.show()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

#### What are the most represented genres?

In [ ]:
# show top 10 genres
df_genre_explode.groupBy("genre").count().orderBy(desc("count")).show(10)

+-------------+-----+
|        genre|count|
+-------------+-----+
|        Indie|34271|
|       Action|23616|
|       Casual|12312|
|    Adventure|11265|
|    Adventure|10166|
|     Strategy|10045|
|       Casual| 9774|
|   Simulation| 9629|
|          RPG| 8719|
| Early Access| 6125|
+-------------+-----+
only showing top 10 rows



#### Do some publishers have favorite genres?

In [ ]:
# show top 50, of number of games made by publisher for each gender
genre_publisher = df_genre_explode.withColumn("publisher", df.data.publisher)
genre_publisher.groupBy("genre", "publisher").count().sort(desc("count")).show(50)

+---------+--------------------+-----+
|    genre|           publisher|count|
+---------+--------------------+-----+
|Adventure|      Big Fish Games|  391|
|   Casual|      Big Fish Games|  389|
|   Casual|              8floor|  194|
|      RPG|     Choice of Games|  136|
|    Indie|     Choice of Games|  112|
|    Indie|        Laush Studio|  106|
|Adventure|     Choice of Games|   99|
|    Indie|                    |   94|
|Adventure|Alawar Entertainment|   94|
|   Casual|Alawar Entertainment|   94|
|    Indie|    Devolver Digital|   81|
|   Action|                SEGA|   80|
|   Casual|        Laush Studio|   79|
|      RPG|        Hosted Games|   79|
|    Indie|  Boogygames Studios|   78|
|      RPG|        Kagura Games|   77|
|    Indie|       Sekai Project|   76|
|    Indie|      Piece Of Voxel|   76|
|   Action|         Square Enix|   75|
|    Indie|Alawar Entertainment|   75|
|    Indie|        Hosted Games|   74|
|Adventure|       Artifex Mundi|   72|
| Strategy|  Boogygames S

### What are the most lucrative genres?

In [ ]:
# ccu = concurrent (active) users
df.select("data.ccu").show(5)

+-----+
|  ccu|
+-----+
|13990|
|    0|
|   99|
|   76|
|    0|
+-----+
only showing top 5 rows



In [ ]:
genre_ccu = df_genre_explode.withColumn("number_users", df.data.ccu)
genre_ccu.show()

+--------------------+-------+-----------+------------+
|                data|     id|      genre|number_users|
+--------------------+-------+-----------+------------+
|{10, [Multi-playe...|     10|     Action|       13990|
|{1000000, [Single...|1000000|     Action|           0|
|{1000000, [Single...|1000000|  Adventure|           0|
|{1000000, [Single...|1000000|      Indie|           0|
|{1000010, [Single...|1000010|  Adventure|          99|
|{1000010, [Single...|1000010|      Indie|          99|
|{1000010, [Single...|1000010|        RPG|          99|
|{1000010, [Single...|1000010|   Strategy|          99|
|{1000030, [Multi-...|1000030|     Action|          76|
|{1000030, [Multi-...|1000030|      Indie|          76|
|{1000030, [Multi-...|1000030| Simulation|          76|
|{1000030, [Multi-...|1000030|   Strategy|          76|
|{1000040, [Single...|1000040|     Action|           0|
|{1000040, [Single...|1000040|     Casual|           0|
|{1000040, [Single...|1000040|      Indie|      

##### Insight
- Top 10 most popular genres:

In [ ]:
# top 10 genres with most users
# altough not necessarily the most profitable,
# there is a natural relationship between popularity and sales
genre_ccu.select("genre", "number_users").groupby("genre")\
    .sum().orderBy(desc("sum(number_users)")).show(10)


+--------------------+-----------------+
|               genre|sum(number_users)|
+--------------------+-----------------+
|              Action|          5678393|
|        Free to Play|          3349267|
|           Adventure|          2362513|
|            Strategy|          1707913|
| Massively Multip...|          1562734|
|                 RPG|          1547343|
|               Indie|          1409669|
|          Simulation|          1056561|
|          Simulation|           451708|
|        Early Access|           412260|
+--------------------+-----------------+
only showing top 10 rows



### Platform Analysis
- Are most games available on Windows/Mac/Linux instead?

- Do certain genres tend to be preferentially available on certain platforms?

### Are most games available on Windows/Mac/Linux instead?

In [ ]:
# value counts for linux
df.groupBy("data.platforms.linux").count().orderBy(desc("count")).show()

+-----+-----+
|linux|count|
+-----+-----+
|false|47233|
| true| 8458|
+-----+-----+



In [ ]:
df.groupBy("data.platforms.mac").count().orderBy(desc("count")).show()

+-----+-----+
|  mac|count|
+-----+-----+
|false|42921|
| true|12770|
+-----+-----+



In [ ]:
df.groupBy("data.platforms.windows").count().orderBy(desc("count")).show()

+-------+-----+
|windows|count|
+-------+-----+
|   true|55676|
|  false|   15|
+-------+-----+



In [ ]:
print("Percentage of games available in Linux:", 8458/ df.count())
print("Percentage of games available in Mac:", 12770/ df.count())
print("Percentage of games available in Windows:", 55676/ df.count())

Percentage of games available in Linux: 0.15187373184176978
Percentage of games available in Mac: 0.22930096424916055
Percentage of games available in Windows: 0.9997306566590652


### Do certain genres tend to be preferentially available on certain platforms?

In [ ]:
# linux
genre_explode_linux = df_genre_explode.withColumn("linux", df.data.platforms.linux,)
genre_explode_linux = genre_explode_linux.filter(genre_explode_linux["linux"] == True)
genre_explode_linux.groupBy("genre").count().orderBy(desc("count")).show(10)

+-----------+-----+
|      genre|count|
+-----------+-----+
|      Indie| 5912|
|     Action| 3369|
|  Adventure| 1945|
|     Casual| 1838|
|   Strategy| 1707|
|     Casual| 1467|
|        RPG| 1438|
| Simulation| 1399|
|  Adventure| 1357|
|      Indie| 1066|
+-----------+-----+
only showing top 10 rows



In [ ]:
# mac
genre_explode_mac = df_genre_explode.withColumn("mac", df.data.platforms.mac,)
genre_mac = genre_explode_mac.filter(genre_explode_mac["mac"] == True)
genre_mac.groupBy("genre").count().orderBy(desc("count")).show(10)

+-----------+-----+
|      genre|count|
+-----------+-----+
|      Indie| 8367|
|     Action| 4549|
|  Adventure| 3150|
|     Casual| 2776|
|   Strategy| 2769|
|     Casual| 2354|
| Simulation| 2191|
|        RPG| 2117|
|  Adventure| 1889|
|      Indie| 1568|
+-----------+-----+
only showing top 10 rows



In [ ]:
# windows
genre_explode_windows = df_genre_explode.withColumn("windows", df.data.platforms.windows,)
genre_explode_windows = genre_explode_windows.filter(genre_explode_windows["windows"] == True)
genre_explode_windows.groupBy("genre").count().orderBy(desc("count")).show(10)

+-------------+-----+
|        genre|count|
+-------------+-----+
|        Indie|34267|
|       Action|23612|
|       Casual|12309|
|    Adventure|11262|
|    Adventure|10165|
|     Strategy|10042|
|       Casual| 9773|
|   Simulation| 9626|
|          RPG| 8718|
| Early Access| 6125|
+-------------+-----+
only showing top 10 rows



##### Insight
- An astonishing 99.97% of games are available on Windows!!! Meanwhile, only 22.93% and 15.19% are available on Mac and Linux
- Movie tastes are very similar for Mac and Linux users, but differ a little more when compared to Windows users. This probably has to do with the fact that Windows users have access to practically all games, while Mac and Linux have access to very few.